In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from feature_engine.outliers import Winsorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df_missing = pd.read_csv('../Data/df_transformed.csv')
df_missing.drop('Unnamed: 0',axis=1,inplace=True)
df_imputed = pd.read_csv('../Data/dfi_transformed.csv')
df_imputed.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
Models = {
    'number':[],
    'description':[],
    'r2':[],
    'adjusted r2':[],
    'training MAE':[],
    'testing MAE':[],
    'training RMSE':[],
    'testing RMSE':[]
}

In [4]:
def model(number,return_model=False):
    global Models,df_missing,df_imputed
    original_number = number
    m = ""
    imputed = False
    num_variables=['RIDAGEYR','BPXPLS','Systolic','Diastolic']
    cat_variables =['RIAGENDR','RIDRETH3']
    if number % 2 == 0: # use imputed values (remember to drop na before train test split)
        df = df_missing
        m = "drop missing, "
    else:
        df = df_imputed
        imputed = True
        m = "imputed, "
        cat_variables.extend(['BPXPLS_i','Systolic_i','Diastolic_i'])
    number = (number - number%2)//2
    if number%2==1: # drop outliers
        m = m + "drop outliers, "
        df = df[df.outlier==0].copy()
    number = (number - number%2)//2
    if number % 2 == 1: # reduce y range
        m = m + "reduce y range, "
        number -= 1
        df = df[(df.LBXTC>=100)&(df.LBXTC<400)].copy()
    number = number//2
    if number % 2 == 1: # use height and weight
        m = m + "use height and weight, "
        number = number - 1
        num_variables.extend([ 'BMXWT', 'BMXHT'])
        if imputed:
            cat_variables.extend(['BMXWT_i', 'BMXHT_i'])
    else:
        m = m + "use bmi, "
        num_variables.append('BMXBMI')
        if imputed:
            cat_variables.append('BMXBMI_i')
    number = number//2
    if number%2==1: # include irregular pulse
        m = m + "include irregular pulse, "
        number -= 1
        cat_variables.append('BPXPULS')
        if imputed:
            cat_variables.append('BPXPULS_i')
    number = number//2
    if number % 2 == 1: # include chronic conditions
        number -= 1
        m = m + "include chronic conditions, "
        cat_variables.extend(['BPQ020', 'BPQ050A', 'BPQ080','BPQ100D', 'DIQ010', 'MCQ170M'])
        if imputed:
            cat_variables.extend(['BPQ020_i', 'BPQ050A_i', 'BPQ080_i', 'BPQ100D_i', 'DIQ010_i','MCQ170M_i'])
    number = number//2
    if number % 2 == 1: # use transformed physical activity
        number -= 1
        m = m + "transformed physical activity,"
        num_variables.append('MET_rec')
        if imputed:
            cat_variables.append('MET_rec_i')
    else:
        num_variables.extend(['PAQ655','PAD660','PAQ670','PAD675'])
        if imputed:
            cat_variables.extend(['PAQ655_i','PAD660_i','PAQ670_i','PAD675_i'])
    number = number // 2
    if number%2 == 0: # don't include nutrients
        number = (number - number%6)//6
    else:
        number = (number-1)//2
        if number % 3 == 0: # include original nutrients
            num_variables.extend(['DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA'])
            cat_variables.append('DR1_300')
            if imputed:
                cat_variables.append('DR1_300_i')
            m = m + "original nutrients included, "
        elif number % 3 == 1: # include transformed nutrients
            m = m + "transformed nutrients included"
            num_variables.extend(['DR1TKCAL_t',
       'DR1TPROT_t', 'DR1TCARB_t', 'DR1TSUGR_t', 'DR1TFIBE_t', 'DR1TTFAT_t',
       'DR1TSFAT_t', 'DR1TMFAT_t', 'DR1TPFAT_t', 'DR1TCHOL_t', 'DR1TSODI_t',
       'DR1TVD_t', 'DR1TCALC_t', 'DR1TIRON_t', 'DR1TPOTA_t'])
            if imputed:
                cat_variables.append('DR1_300_i')
        else: # include both
            m = m + "both nutrients included, "
            num_variables.extend(['DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA','DR1TKCAL_t',
       'DR1TPROT_t', 'DR1TCARB_t', 'DR1TSUGR_t', 'DR1TFIBE_t', 'DR1TTFAT_t',
       'DR1TSFAT_t', 'DR1TMFAT_t', 'DR1TPFAT_t', 'DR1TCHOL_t', 'DR1TSODI_t',
       'DR1TVD_t', 'DR1TCALC_t', 'DR1TIRON_t', 'DR1TPOTA_t'])
            if imputed:
                cat_variables.append('DR1_300_i')
        number = (number - number%3)//3
    if number % 2 == 0: # don't include alcohol and tobacco
        number = (number - number % 6)//6
    else: # include alcohol and tobacco
        number = (number - 1)//2
        # add tobacco here
        cat_variables.append('SMQ681')
        if imputed:
            cat_variables.append('SMQ681_i')
        if number % 3 == 0: # include original alcohol quantities
            m = m + "tobacco and alcohol questionairre variables"
            num_variables.extend([ 'ALQ120Q','ALQ130'])
            cat_variables.append('ALQ120U')
            if imputed:
                cat_variables.extend(['ALQ120Q_i', 'ALQ120U_i', 'ALQ130_i'])
        elif number % 3 == 1: # include transformed alcohol quantities
            m = m + "tobacco and average alcohol, "
            num_variables.append('avgALC')
            if imputed:
                cat_variables.append('avgALC_i')
        else: # include both
            m = m + "tobacco and both alcohol variables"
            num_variables.extend([ 'ALQ120Q','ALQ130','avgALC'])
            cat_variables.append('ALQ120U')
            if imputed:
                cat_variables.extend(['ALQ120Q_i', 'ALQ120U_i', 'ALQ130_i','avgALC_i'])
        number = (number-number%3)//3
    df = df[num_variables+cat_variables+['LBXTC','log_LBXTC']].copy()
    df.dropna(inplace=True)
    y = df.LBXTC
    if number % 2 == 1: # use log transformed y
        m = m + "log transform y"
        number -= 1
        y = df.log_LBXTC
    number = number//2
    X = df[num_variables+cat_variables].copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=675, test_size=0.2)
    numeric_transformer = Pipeline(steps=[('scalar',StandardScaler())])
    numeric_transformer.steps.pop(0)
    cat_transformer = Pipeline(steps=[('one_hot',OneHotEncoder(handle_unknown='ignore'))])
    if number % 2 == 1: # Winsorize x
        m = m + "Winsorize x, "
        numeric_transformer.steps.insert(0,('winsorizor',Winsorizer()))
        number -= 1
    number = number//2
    if number % 2 == 1: # log transform x
        m = m + "Log transform x values, "
        number -= 1
        numeric_transformer.steps.insert(0,('log transform x',FunctionTransformer(np.log1p)))
    number = number // 2
    if number%2==1: # add a scaler
        number -= 1
        m = m + "scale values, "
        numeric_transformer.steps.append(('scalar',StandardScaler()))
    number = number//2
    preprocessor = ColumnTransformer(transformers=[
        ('num',numeric_transformer,num_variables),
        ('cat',cat_transformer,cat_variables)])
    gb1 = Pipeline(steps =[
        ('preprocessor',preprocessor),
        ('gb', GradientBoostingRegressor())])
    if number % 2 == 1: # add PCA
        gb1.steps.insert(1,("pca",PCA(0.95)))
        m = m + "added PCA"

    gb1.fit(X_train,y_train)
    if return_model:
        return (gb1, X_train, X_test, y_train, y_test)
    Models['number'].append(original_number)
    Models['description'].append(m)
    r2 = gb1.score(X_train,y_train)
    Models['r2'].append(r2)
    k = gb1.named_steps['gb'].n_features_in_
    #n = rf1.named_steps['rf'].n_samples_fit_
    # may need to fix this as no n samples fit and pca may reduce features
    n = X_train.shape[0]
    ar2 = ((1-r2)/(n-1))/(n-k-1)
    Models['adjusted r2'].append(ar2)
    y_train_preds = gb1.predict(X_train)
    y_preds = gb1.predict(X_test)
    Models['training MAE'].append(mean_absolute_error(y_train,y_train_preds))
    Models['testing MAE'].append(mean_absolute_error(y_test,y_preds))
    Models['training RMSE'].append(mean_squared_error(y_train,y_train_preds,squared=False))
    Models['testing RMSE'].append(mean_squared_error(y_test,y_preds,squared=False))

In [5]:
model_numbers = []
for i in range(20):
    n = np.random.randint(2*73728)
    while n in model_numbers:
        n = np.random.randint(2*73728)
    model_numbers.append(n)
    try:
        model(n)
    except:
        print(n)
models = pd.DataFrame(Models)
models.sort_values(by='adjusted r2',ascending=False)

76030
73980


,number,description,r2,adjusted r2,training MAE,testing MAE,training RMSE,testing RMSE
1,68552,"drop missing, use height and weight, transform...",0.314633,8.394075e-08,26.411665,30.735214,34.497119,40.912071
14,73010,"drop missing, drop outliers, use bmi, include ...",0.342991,8.262190e-08,0.137453,0.148569,0.177174,0.186858
9,133548,"drop missing, reduce y range, use height and w...",0.352410,8.204466e-08,25.230936,30.958461,31.991965,39.124210
15,105850,"drop missing, drop outliers, use height and we...",0.359756,7.980104e-08,26.101345,29.068902,33.743195,37.340593
3,121328,"drop missing, use bmi, include irregular pulse...",0.371379,7.868632e-08,25.871597,29.160378,33.275859,38.702056
12,10478,"drop missing, drop outliers, reduce y range, u...",0.404368,7.669007e-08,23.952091,28.846509,30.596341,36.850738
2,36852,"drop missing, reduce y range, use bmi, include...",0.412123,7.561499e-08,0.126852,0.149775,0.159690,0.189538
10,69143,"imputed, drop outliers, reduce y range, use bm...",0.211413,5.879024e-08,0.148805,0.159620,0.187480,0.201071
16,48066,"drop missing, drop outliers, use bmi, transfor...",0.263277,5.733953e-08,27.830474,29.558191,36.438671,37.746970
11,127767,"imputed, drop outliers, reduce y range, use bm...",0.229420,5.705637e-08,0.147419,0.165877,0.185327,0.207476
